In [ ]:
import sys
import csv
import math
import copy
import time
import numpy as np
from collections import Counter
from numpy import *
import pandas as pd
from random import shuffle

In [ ]:
feature = []

In [ ]:
def normalize(matrix):
  a = np.array(matrix)
  a = a.astype(float)
  normalized_matrix = (a - a.min(axis = 0)) / (a.max(axis = 0) - a.min(axis = 0))
  return normalized_matrix

In [ ]:
def load_csv(file):
  df = pd.read_csv(file)
  dataframe = df.to_numpy()
  return dataframe

In [ ]:
def random_numpy_array(arr):
  np.random.shuffle(arr)
  return arr

In [ ]:
def entropy(y):
    class_frequency = {}
    attribute_entropy = 0

    for i in y:
        if i in class_frequency:
            class_frequency[i]+=1
        else:
            class_frequency[i] = 1

    for x in class_frequency.values():
        p1= -x/float(len(y))
        p2= math.log(x/float(len(y)),2)
        attribute_entropy +=  p1*p2 

    return attribute_entropy

In [ ]:
def generate_set(X):
  Y = X[:,-1]
  Y = Y.reshape(len(Y),1)
  normalized_new_X = normalize(X[:,:-1])
  X = np.concatenate((normalized_new_X,Y),axis=1)
  rows= X.shape[0]
  testing_set = round(rows * 0.1)

  start = 0
  end = testing_set

  testing_features = []
  testing_labels = []
  training_features = []
  training_labels = []

  for i in range(10):
    test_set = X[start:end,:]
    train_beforetest = X[:start,:]
    train_aftertest = X[end:,:]
    train_set = np.concatenate((train_beforetest,train_aftertest),axis=0)
    testing_set_labels = test_set[:,-1]
    testing_set_labels.flatten()
    train_set_labels = train_set[:,-1]
    train_set_labels.flatten()
    test_set = test_set[:,:-1].astype(np.float)
    train_set = train_set[:,:-1].astype(np.float)
    testing_features.append(test_set)
    testing_labels.append(testing_set_labels)
    training_features.append(train_set)
    training_labels.append(train_set_labels)
    start = end
    end = end + testing_set

  return training_features, training_labels, testing_features, testing_labels

In [ ]:
class Node(object):
  def __init__(self, val, lchild, rchild,thea,leaf):
    self.root_value = val
    self.root_left = lchild
    self.root_right = rchild
    self.theta = thea
    self.leaf = leaf

  def is_leaf(self):
    return self.leaf

  def ret_thetha(self):
    return self.theta

  def ret_root_value(self):
    return self.root_value

  def ret_llist(self):
    return self.root_left

  def ret_rlist(self):
    return self.root_right

  def __repr__(self):
    return "(%r, %r, %r, %r)" %(self.root_value,self.root_left,self.root_right,self.theta)

In [ ]:
class DecisionTree(object):
  fea_list = []

  def __init__(self):
    self.root_node = None

  def cal_major_class_values(self,class_values):
    return Counter(class_values).most_common(1)[0][0]

  def cal_best_theta_value(self,ke,attri_list):
    data_list = []
    class_labels= []

    for i in attri_list:
      data_list.append(i[0])
      class_labels.append(i[1])
    entropy_result = entropy(class_labels)
    max_info_gain = 0
    theta=0

    best_index_left = []
    best_index_right = []
    class_labels_AS = []
    data_list.sort()

    for i in range(len(data_list)-1):
      current_theta = float(data_list[i]+data_list[i+1])/2

      index_less_than = []
      value_less_than = []
      index_greater_than = []
      value_greater_than =[]
      counter = 0

      for c,j in enumerate(attri_list):
        if (j[0]<=current_theta):
          index_less_than.append(c)
          value_less_than.append(j[1])
        else:
          index_greater_than.append(c)
          value_greater_than.append(j[1]) 
      entropy_less= entropy(value_less_than)
      entropy_great= entropy(value_greater_than)
      info_gain= (len(index_less_than)/len(class_labels)) * entropy_less + ((len(index_greater_than)/len(class_labels))*entropy_great)
      info_gain = entropy_result -info_gain

      if info_gain> max_info_gain:
        max_info_gain=info_gain
        theta= current_theta
        best_index_right=index_greater_than
        best_index_left=index_less_than
        class_labels_AS= value_greater_than+value_less_than

    return max_info_gain, theta, best_index_left, best_index_right, class_labels_AS

  def best_feature(self,dict_rep):
    key_value=None

    best_info_gain=-1
    best_theta=0
    best_left_list=[]
    best_right_rist=[]
    best_class_labels_AS=[]
    result_list=[]

    for ke in dict_rep.keys():
      max_info_gain, theta, best_index_left,best_index_right, class_labels_AS= self.cal_best_theta_value(ke, dict_rep[ke])
            
      if max_info_gain> best_info_gain:
        best_info_gain=max_info_gain
        best_theta=theta
        key_value= ke
        best_left_list=best_index_left
        best_right_rist=best_index_right
        best_class_labels_AS= class_labels_AS
    result_list.append(key_value)
    result_list.append(best_theta)
    result_list.append(best_left_list)
    result_list.append(best_right_rist)
    result_list.append(best_class_labels_AS)
    return result_list

  def get_remainder_dict(self,dict_of_everything,index_split):
    global fea_list
    split_dict={}
    for ke in dict_of_everything.keys():
      value_list=[]
      modified_list=[]
      key_values= dict_of_everything[ke]
      for each_value in range(len(key_values)):
        if each_value not in index_split:
          modified_list.append(key_values[each_value])
          value_list.append(key_values[each_value][1])
      split_dict[ke]=modified_list
    return split_dict, value_list

  def create_decision_tree(self, dict_of_everything,class_val,eta_min_val):
    global fea_list
    if len(set(class_val)) ==1:
      return Node(class_val[0],None,None,0,True)
    elif len(class_val) < eta_min_val:
      class_label = self.cal_major_class_values(class_val)
      return Node(class_label,None,None,0,True)
    else:
      best_feature_list = self.best_feature(dict_of_everything)
      node_name = best_feature_list[0]
      theta = best_feature_list[1]
      left_split = best_feature_list[2]
      right_split = best_feature_list[3]
      labels = best_feature_list[4]

    left_split_dict, left_value_list= self.get_remainder_dict(dict_of_everything, left_split)
    right_split_dict, right_value_list= self.get_remainder_dict(dict_of_everything, right_split)
    left_split_node = self.create_decision_tree(left_split_dict,left_value_list,eta_min_val)
    right_split_node = self.create_decision_tree(right_split_dict, right_value_list, eta_min_val)
    root_node= Node(node_name, right_split_node, left_split_node, theta, False)
    return root_node

  def fit(self, dict_of_everything,cl_val,features,eta_min_val):
      global fea_list
      fea_list=features
      self.root_node= self.create_decision_tree(dict_of_everything,cl_val,eta_min_val)

      return self.root_node

  def classify(self,row,root):
    current_node = root
    while not current_node.leaf:
      if row[current_node.root_value]<current_node.theta:
        current_node  = current_node.root_left
      else:
        current_node = current_node.root_right
    return current_node.root_value

  def predict(self, X, root):
    prediction_list=[]
    for i in X:
      prediction_list.append(self.classify(i , root))
    return prediction_list

In [ ]:
def accuracy_for_predicted_values(test_class_names1,l):
  true_count= 0
  false_count = 0

  for i in range(len(test_class_names1)):
    if test_class_names1[i] == l[i]:
      true_count += 1
    else:
      false_count += 1
  
  return true_count / (true_count + false_count)

In [ ]:
def build_dict_of_attributes_with_class_values(X,y):
  attribute_dict = {}
  feature_list = []

  for i in range(X.shape[1]):
    feature_index = i
    values = X[:,i]
    attribute_list = []
    counter = 0
    for value in values:
      attribute_value = []
      attribute_value.append(value)
      attribute_value.append(y[counter])
      attribute_list.append(attribute_value)
      counter+=1
    attribute_dict[feature_index] = attribute_list
    feature_list.append(feature_index)
  return attribute_dict,feature_list

In [ ]:
def main(num_arr, eta_min):
  eta_min_val = round(eta_min*num_arr.shape[0])
  num_arr = random_numpy_array(num_arr)

  training_features,training_labels, testing_features, testing_labels = generate_set(num_arr)

  cumulate_acc = 0

  for i in range(10):
    attribute_dict,feature_list = build_dict_of_attributes_with_class_values(training_features[i],training_labels[i])

    DTree = DecisionTree()
    DTree.fit(attribute_dict,training_labels[i],feature_list,eta_min_val)
        
    predicted_labels = DTree.predict(testing_features[i],DTree.root_node)

    accu = accuracy_for_predicted_values(predicted_labels,testing_labels[i])
    cumulate_acc+=accu
    print("Accuracy is ",accu)
  
  print("Accuracy across 10-cross validation for",eta_min,"is",float(cumulate_acc)/10)

In [ ]:
eta_min_list = [0.05,0.10,0.15,0.20]
newfile = "iris.csv"
num_arr = load_csv(newfile)
for i in eta_min_list:
    main(num_arr,i)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Accuracy is  1.0
Accuracy is  0.9333333333333333
Accuracy is  0.8666666666666667
Accuracy is  1.0
Accuracy is  0.9333333333333333
Accuracy is  1.0
Accuracy is  0.9333333333333333
Accuracy is  0.9333333333333333
Accuracy is  1.0
Accuracy is  0.9333333333333333
Accuracy across 10-cross validation for 0.05 is 0.9533333333333335
Standard Deviation:   0.04268749491621898
Accuracy is  1.0
Accuracy is  0.8666666666666667
Accuracy is  1.0
Accuracy is  1.0
Accuracy is  0.8666666666666667
Accuracy is  0.9333333333333333
Accuracy is  0.9333333333333333
Accuracy is  1.0
Accuracy is  1.0
Accuracy is  1.0
Accuracy across 10-cross validation for 0.1 is 0.9600000000000002
Standard Deviation:   0.05333333333333332
Accuracy is  0.8666666666666667
Accuracy is  1.0
Accuracy is  1.0
Accuracy is  1.0
Accuracy is  1.0
Accuracy is  0.9333333333333333
Accuracy is  0.9333333333333333
Accuracy is  1.0
Accuracy is  0.9333333333333333
Accuracy is  0.8
Accuracy across 10-cross validation for 0.15 is 0.9466666666666